In [10]:
import os
import sys
sys.path.append("..")
from base_fns import get_local_folder
os.chdir(os.path.dirname(get_local_folder()))

from tqdm.auto import tqdm
import pandas as pd
import tabulate as tb
from hydra import compose, initialize
from omegaconf import OmegaConf
import random
from munch import Munch
import numpy as np
from script.ct_model import CtDataset, CtDataModule,CtModel, CtTagger
from script.eval import evaluate
import torch
# from transformers import BertTokenizerFast as BertTokenizer
from transformers import AutoTokenizer
from torchmetrics import F1Score, Accuracy
from sklearn.metrics import classification_report, precision_score

import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping


with initialize(
    version_base=None,
    config_path="../cfg",
):
    cfg = compose(config_name="main")
    
np.random.seed(cfg.random.seed)
random.seed(cfg.random.seed)


# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [11]:
LABELS=cfg.model.labels
MAX_TOKEN_COUNT = cfg.model.max_token_count
THRESHOLD = cfg.model.threshold
TEST_FP = cfg.data.processed.text.test
test_df = pd.read_json(TEST_FP, lines=True)

In [12]:
results = {}

stub_dict = {"train_silver" : "silver",
             "train_bronze" : "bronze",
             "train_ml_ab": "ml_ab",
             "fulltext": "ft",
             "abstract": "abs"
             }


for dataset, v in cfg.checkpoint.items():
    for text_col, v in cfg.checkpoint[dataset].items():
        TEXT_COL = cfg.model.text_col[text_col]
        for lm, ckpt in cfg.checkpoint[dataset][text_col].items(): 

            stub = f"{stub_dict[dataset]}/{stub_dict[text_col]}/{lm}"
            print(f"<<{stub}>>")
            CHECKPOINT_PATH = ckpt
            LM_MODEL_NAME = cfg.model.lm[lm]
            tagger = CtTagger(CHECKPOINT_PATH, LABELS, TEXT_COL, LM_MODEL_NAME)
            predictions, labels = tagger.predict(test_df)
            y_pred_score = predictions.numpy().flatten()
            y_true = labels.numpy().flatten()
            results[stub] = evaluate(y_true, y_pred_score)



<<ml_ab/ft/pmbert>>


100%|██████████| 600/600 [00:10<00:00, 58.78it/s]


In [13]:
RESULTS_FP = cfg.evaluation.model_eval.results_fp
df =pd.DataFrame.from_dict(results, orient='index')
df

,precision,recall,F1,P10,P100,P200
ml_ab/ft/pmbert,0.5665,0.98,0.7179,1.0,0.97,0.88


In [ ]:
with open(RESULTS_FP, "w") as f:
    json_str = df.to_json(orient='records', lines=True)
    f.write(json_str)